In [1]:
!pip install yfinance tqdm pytz fredapi --quiet

In [2]:
import yfinance as yf
import pandas as pd
from tqdm import tqdm
import pytz

### SPY 주식 데이터 가져오기(고가, 저가, 종가, 조정종가, 변동성, 거래량)

In [3]:
# ============================================
# 수집할 종목 리스트 정의
# ============================================
symbols = ["SPY"]

In [4]:
# =============================
# 2. 시계열 데이터 수집 (뉴욕시간 기준)
# =============================
START = "2022-11-07"
END   = "2025-11-09"   # end는 제외 → 11/07까지 포함

In [5]:
print("[1/2] 시계열 데이터 수집 중 ...")
data = yf.download(symbols, start=START, end=END, interval="1d", auto_adjust=False)

[1/2] 시계열 데이터 수집 중 ...


[*********************100%***********************]  1 of 1 completed


In [6]:
# UTC로 지정 → 뉴욕시간으로 변환
data.index = pd.to_datetime(data.index, utc=True).tz_convert("America/New_York")

# '세션 날짜'를 계산: 전날 저녁 라벨 → 다음날 거래일로 이동
session_dates = (data.index + pd.Timedelta(days=1)).normalize()

# '09:30' (개장시각) 타임스탬프로 교체
session_times = session_dates + pd.Timedelta(hours=9, minutes=30)

# 이걸 인덱스로 지정
data.index = session_times

print("인덱스(라벨)를 실제 뉴욕 개장 시각(09:30)으로 조정 완료")
print(data.index.min(), "→", data.index.max())

인덱스(라벨)를 실제 뉴욕 개장 시각(09:30)으로 조정 완료
2022-11-07 09:30:00-05:00 → 2025-11-07 09:30:00-05:00


In [7]:
# level0 컬럼 중 'Adj Close' → 'AdjClose' (공백 제거)
data = data.rename(columns={'Adj Close':'AdjClose'}, level=0)

print("데이터 레벨명:", data.columns.names)   # ['Attributes', 'Symbols'] 형태
print("기간:", data.index.min().date(), "~", data.index.max().date())

데이터 레벨명: ['Price', 'Ticker']
기간: 2022-11-07 ~ 2025-11-07


In [8]:
# 필요한 컬럼 정리
ohlcv = data[["High", "Low", "Close", "Volume"]].copy()
ohlcv.columns = [f"{ticker}_{col}" for col, ticker in ohlcv.columns]

In [9]:
# 변동성(High-Low)
volatility = pd.DataFrame(index=ohlcv.index)
for sym in symbols:
    h, l = f"{sym}_High", f"{sym}_Low"
    if h in ohlcv.columns and l in ohlcv.columns:
        volatility[f"{sym}_Volatility"] = ohlcv[h] - ohlcv[l]

price_features = pd.concat([ohlcv, volatility], axis=1).sort_index()

In [ ]:
price_features.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 754 entries, 2022-11-07 09:30:00-05:00 to 2025-11-07 09:30:00-05:00
Data columns (total 5 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   SPY_High        754 non-null    float64
 1   SPY_Low         754 non-null    float64
 2   SPY_Close       754 non-null    float64
 3   SPY_Volume      754 non-null    int64  
 4   SPY_Volatility  754 non-null    float64
dtypes: float64(4), int64(1)
memory usage: 35.3 KB


In [10]:
price_features

,SPY_High,SPY_Low,SPY_Close,SPY_Volume,SPY_Volatility
Date,,,,,
2022-11-07 09:30:00-05:00,380.570007,375.529999,379.950012,68286900,5.040009
2022-11-08 09:30:00-05:00,385.119995,377.720001,382.000000,84641100,7.399994
2022-11-09 09:30:00-05:00,381.140015,373.609985,374.130005,78495500,7.530029
2022-11-10 09:30:00-05:00,395.040009,385.640015,394.690002,141455800,9.399994
2022-11-11 09:30:00-05:00,399.350006,393.609985,398.510010,93839900,5.740021
...,...,...,...,...,...
2025-11-03 09:30:00-05:00,685.799988,679.940002,683.340027,57315000,5.859985
2025-11-04 09:30:00-05:00,679.960022,674.580017,675.239990,78427000,5.380005
2025-11-05 09:30:00-05:00,680.859985,674.169983,677.580017,74402400,6.690002


In [28]:
# CSV 저장
price_features.to_csv("../raw_data/SPY_주식데이터.csv")

### 환경변수 데이터 가져오기(단기채, 유가, 고용자수, 실업률 추가)

In [20]:
import pandas as pd
import yfinance as yf
from fredapi import Fred

# ============================================
# 공통 설정
# ============================================
FRED_API_KEY = "0cfb957db66cb7dd8622b000bf3941db"
fred = Fred(api_key=FRED_API_KEY)

START = "2022-01-01"
END   = "2025-11-08"   # end는 제외 → 11/07까지 포함

symbols = ['^VIX','DX-Y.NYB','TLT','GC=F','GLD','SHY']


# ============================================
# 1) yfinance: 가격 데이터 수집 + 가공
# ============================================
def get_price_features(symbols, start, end):
    print("[1/2] yfinance 시계열 데이터 수집 중 ...")
    data = yf.download(symbols, start=start, end=end, interval="1d", auto_adjust=False)

    # 인덱스를 UTC로 지정 → 뉴욕시간으로 변환
    data.index = pd.to_datetime(data.index, utc=True).tz_convert("America/New_York")

    # '세션 날짜'를 계산: 전날 저녁 라벨 → 다음날 거래일로 이동
    session_dates = (data.index + pd.Timedelta(days=1)).normalize()

    # '09:30' (개장시각) 타임스탬프로 교체
    session_times = session_dates + pd.Timedelta(hours=9, minutes=30)

    # 이걸 인덱스로 지정
    data.index = session_times

    print("인덱스(라벨)를 실제 뉴욕 개장 시각(09:30)으로 조정 완료")
    print(data.index.min(), "→", data.index.max())

    # # level0 컬럼 중 'Adj Close' → 'AdjClose' (공백 제거)
    # data = data.rename(columns={'Adj Close': 'AdjClose'}, level=0)

    print("데이터 레벨명:", data.columns.names)   # ['Attributes', 'Symbols'] 형태
    print("기간:", data.index.min().date(), "~", data.index.max().date())

    # 필요한 컬럼 정리 (OHLCV + 변동성)
    ohlcv = data[["High", "Low", "Close", "Volume"]].copy()
    ohlcv.columns = [f"{ticker}_{col}" for col, ticker in ohlcv.columns]

    # 변동성(High-Low)
    volatility = pd.DataFrame(index=ohlcv.index)
    for sym in symbols:
        h, l = f"{sym}_High", f"{sym}_Low"
        if h in ohlcv.columns and l in ohlcv.columns:
            volatility[f"{sym}_Volatility"] = ohlcv[h] - ohlcv[l]

    price_features = pd.concat([ohlcv, volatility], axis=1).sort_index()

    return price_features


# ============================================
# 2) FRED: 거시지표 + GDP 수집 + 정렬
# ============================================
def get_macro_features(start, end, target_index):
    """
    start, end: FRED observation 범위
    target_index: price_features.index (뉴욕 09:30, 거래일 기준)
    """
    print("[2/2] FRED 거시지표 수집 중 ...")

    # 2-1) WTI 현물 spot 가격 (DCOILWTICO, 일별)
    wti = fred.get_series(
        "DCOILWTICO",
        observation_start=start,
        observation_end=end
    ).to_frame("WTI_Spot")

    # 2-2) PAYEMS (비농업 고용자 수, 월별)
    payems = fred.get_series(
        "PAYEMS",
        observation_start=start,
        observation_end=end
    ).to_frame("PAYEMS")

    # 2-3) UNRATE (실업률, 월별)
    unrate = fred.get_series(
        "UNRATE",
        observation_start=start,
        observation_end=end
    ).to_frame("UNRATE")

    # # 2-4) US_GDP_Real (실질 GDP, 분기별, GDPC1)
    # gdp = fred.get_series(
    #     "GDPC1",
    #     observation_start=start,
    #     observation_end=end
    # ).to_frame("US_GDP_Real")

    # 인덱스를 datetime으로 통일
    # for df in [wti, payems, unrate, gdp]:
    for df in [wti, payems, unrate]:
        df.index = pd.to_datetime(df.index)

    # 하나로 합치기
    macro = pd.concat([wti, payems, unrate], axis=1)
    # macro = pd.concat([wti, payems, unrate, gdp], axis=1)

    # FRED 인덱스를 뉴욕 09:30으로 맞추기
    macro.index = macro.index.tz_localize("America/New_York") + pd.Timedelta(hours=9, minutes=30)

    # price_features.index(=뉴욕 09:30, 거래일 기준)에 맞춰 재인덱싱 + ffill
    macro_aligned = macro.reindex(target_index).ffill()

    return macro_aligned


# ============================================
# 3) 메인: 둘을 결합
# ============================================
price_features = get_price_features(symbols, START, END)
macro_aligned  = get_macro_features(START, END, price_features.index)

final_df = price_features.join(macro_aligned, how="left")

print("최종 컬럼 목록:")
print(final_df.columns)

print("\n샘플 데이터:")
print(final_df.head(10))


[*********************100%***********************]  6 of 6 completed

[1/2] yfinance 시계열 데이터 수집 중 ...
인덱스(라벨)를 실제 뉴욕 개장 시각(09:30)으로 조정 완료
2022-01-03 09:30:00-05:00 → 2025-11-07 09:30:00-05:00
데이터 레벨명: ['Price', 'Ticker']
기간: 2022-01-03 ~ 2025-11-07
[2/2] FRED 거시지표 수집 중 ...


최종 컬럼 목록:
Index(['DX-Y.NYB_High', 'GC=F_High', 'GLD_High', 'SHY_High', 'TLT_High',
       '^VIX_High', 'DX-Y.NYB_Low', 'GC=F_Low', 'GLD_Low', 'SHY_Low',
       'TLT_Low', '^VIX_Low', 'DX-Y.NYB_Close', 'GC=F_Close', 'GLD_Close',
       'SHY_Close', 'TLT_Close', '^VIX_Close', 'DX-Y.NYB_Volume',
       'GC=F_Volume', 'GLD_Volume', 'SHY_Volume', 'TLT_Volume', '^VIX_Volume',
       '^VIX_Volatility', 'DX-Y.NYB_Volatility', 'TLT_Volatility',
       'GC=F_Volatility', 'GLD_Volatility', 'SHY_Volatility', 'WTI_Spot',
       'PAYEMS', 'UNRATE'],
      dtype='object')

샘플 데이터:
                           DX-Y.NYB_High    GC=F_High    GLD_High   SHY_High  \
Date                                                                           
2022-01-03 09:30:00-05:00      96.330002  1830.099976  169.009995  85.459999   
2022-01-04 09:30:00-05:00      96.459999  1815.300049  169.720001  85.489998   
2022-01-05 09:30:00-05:00      96.330002  1824.599976  170.929993  85.459999   
2022-01-06 09:30:00-05:00  

In [21]:
final_df.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 970 entries, 2022-01-03 09:30:00-05:00 to 2025-11-07 09:30:00-05:00
Data columns (total 33 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   DX-Y.NYB_High        969 non-null    float64
 1   GC=F_High            969 non-null    float64
 2   GLD_High             967 non-null    float64
 3   SHY_High             967 non-null    float64
 4   TLT_High             967 non-null    float64
 5   ^VIX_High            967 non-null    float64
 6   DX-Y.NYB_Low         969 non-null    float64
 7   GC=F_Low             969 non-null    float64
 8   GLD_Low              967 non-null    float64
 9   SHY_Low              967 non-null    float64
 10  TLT_Low              967 non-null    float64
 11  ^VIX_Low             967 non-null    float64
 12  DX-Y.NYB_Close       969 non-null    float64
 13  GC=F_Close           969 non-null    float64
 14  GLD_Close            967 non-null    floa

In [22]:
final_df

,DX-Y.NYB_High,GC=F_High,GLD_High,SHY_High,TLT_High,^VIX_High,DX-Y.NYB_Low,GC=F_Low,GLD_Low,SHY_Low,...,^VIX_Volume,^VIX_Volatility,DX-Y.NYB_Volatility,TLT_Volatility,GC=F_Volatility,GLD_Volatility,SHY_Volatility,WTI_Spot,PAYEMS,UNRATE
Date,,,,,,,,,,,,,,,,,,,,,
2022-01-03 09:30:00-05:00,96.330002,1830.099976,169.009995,85.459999,146.940002,18.540001,95.629997,1798.800049,168.000000,85.419998,...,0.0,1.980001,0.700005,2.660004,31.299927,1.009995,0.040001,75.99,NaN,NaN
2022-01-04 09:30:00-05:00,96.459999,1815.300049,169.720001,85.489998,144.130005,17.809999,96.029999,1800.000000,168.729996,85.440002,...,0.0,1.469999,0.430000,1.389999,15.300049,0.990005,0.049995,77.00,NaN,NaN
2022-01-05 09:30:00-05:00,96.330002,1824.599976,170.929993,85.459999,144.160004,20.170000,95.889999,1813.099976,168.899994,85.370003,...,0.0,3.590000,0.440002,1.449997,11.500000,2.029999,0.089996,77.83,NaN,NaN
2022-01-06 09:30:00-05:00,96.389999,1791.300049,167.750000,85.339996,143.440002,21.059999,96.040001,1787.099976,166.860001,85.290001,...,0.0,1.980000,0.349998,1.150009,4.200073,0.889999,0.049995,79.47,NaN,NaN
2022-01-07 09:30:00-05:00,96.300003,1797.000000,168.009995,85.320000,143.240005,20.799999,95.709999,1784.400024,166.860001,85.260002,...,0.0,2.230000,0.590004,1.680008,12.599976,1.149994,0.059998,79.00,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2025-11-03 09:30:00-05:00,99.989998,4020.000000,370.839996,82.750000,89.820000,18.830000,99.709999,3959.000000,367.540009,82.709999,...,0.0,1.830000,0.279999,0.400002,61.000000,3.299988,0.040001,61.79,159540.0,4.3
2025-11-04 09:30:00-05:00,100.260002,3995.399902,365.739990,82.779999,90.059998,20.480000,99.739998,3927.399902,361.390015,82.750000,...,0.0,2.580000,0.520004,0.320000,68.000000,4.349976,0.029999,61.38,159540.0,4.3
2025-11-05 09:30:00-05:00,100.360001,3983.500000,367.100006,82.760002,89.550003,20.020000,100.059998,3929.899902,364.649994,82.680000,...,0.0,2.710001,0.300003,0.670006,53.600098,2.450012,0.080002,60.40,159540.0,4.3


In [23]:
import datetime as dt

start_date = dt.date(2022, 11, 7)
end_date   = dt.date(2025, 11, 7)

mask = (
    (final_df.index.date >= start_date) &
    (final_df.index.date <= end_date)
)

final_df = final_df.loc[mask].copy()

print(final_df.index.min(), "→", final_df.index.max())
print(len(final_df))


2022-11-07 09:30:00-05:00 → 2025-11-07 09:30:00-05:00
757


In [24]:
final_df

,DX-Y.NYB_High,GC=F_High,GLD_High,SHY_High,TLT_High,^VIX_High,DX-Y.NYB_Low,GC=F_Low,GLD_Low,SHY_Low,...,^VIX_Volume,^VIX_Volatility,DX-Y.NYB_Volatility,TLT_Volatility,GC=F_Volatility,GLD_Volatility,SHY_Volatility,WTI_Spot,PAYEMS,UNRATE
Date,,,,,,,,,,,,,,,,,,,,,
2022-11-07 09:30:00-05:00,111.269997,1677.900024,156.570007,80.639999,94.860001,25.670000,110.050003,1672.900024,155.809998,80.599998,...,0.0,1.330000,1.219994,1.620003,5.000000,0.760010,0.040001,91.80,154210.0,3.6
2022-11-08 09:30:00-05:00,110.610001,1712.099976,159.869995,80.709999,94.959999,26.160000,109.370003,1703.199951,155.979996,80.639999,...,0.0,1.920000,1.239998,1.190002,8.900024,3.889999,0.070000,88.80,154210.0,3.6
2022-11-09 09:30:00-05:00,110.639999,1720.199951,160.350006,80.839996,95.019997,26.350000,109.449997,1706.000000,158.449997,80.699997,...,0.0,1.330000,1.190002,1.329994,14.199951,1.900009,0.139999,85.79,154210.0,3.6
2022-11-10 09:30:00-05:00,110.989998,1750.300049,163.509995,81.290001,98.430000,26.590000,107.709999,1708.199951,161.660004,81.160004,...,0.0,3.750000,3.279999,1.790001,42.100098,1.849991,0.129997,86.52,154210.0,3.6
2022-11-11 09:30:00-05:00,108.440002,1766.000000,164.580002,81.279999,98.370003,23.910000,106.279999,1757.599976,163.419998,81.169998,...,0.0,1.539999,2.160004,0.830002,8.400024,1.160004,0.110001,89.14,154210.0,3.6
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2025-11-03 09:30:00-05:00,99.989998,4020.000000,370.839996,82.750000,89.820000,18.830000,99.709999,3959.000000,367.540009,82.709999,...,0.0,1.830000,0.279999,0.400002,61.000000,3.299988,0.040001,61.79,159540.0,4.3
2025-11-04 09:30:00-05:00,100.260002,3995.399902,365.739990,82.779999,90.059998,20.480000,99.739998,3927.399902,361.390015,82.750000,...,0.0,2.580000,0.520004,0.320000,68.000000,4.349976,0.029999,61.38,159540.0,4.3
2025-11-05 09:30:00-05:00,100.360001,3983.500000,367.100006,82.760002,89.550003,20.020000,100.059998,3929.899902,364.649994,82.680000,...,0.0,2.710001,0.300003,0.670006,53.600098,2.450012,0.080002,60.40,159540.0,4.3


In [25]:
# drop 사용
final_df = final_df.drop(columns=['DX-Y.NYB_High', 'GC=F_High', 'GLD_High', 'SHY_High', 'TLT_High',
       '^VIX_High', 'DX-Y.NYB_Low', 'GC=F_Low', 'GLD_Low', 'SHY_Low',
       'TLT_Low', '^VIX_Low','DX-Y.NYB_Volume','^VIX_Volume'])

In [26]:
final_df.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 757 entries, 2022-11-07 09:30:00-05:00 to 2025-11-07 09:30:00-05:00
Data columns (total 19 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   DX-Y.NYB_Close       756 non-null    float64
 1   GC=F_Close           756 non-null    float64
 2   GLD_Close            754 non-null    float64
 3   SHY_Close            754 non-null    float64
 4   TLT_Close            754 non-null    float64
 5   ^VIX_Close           754 non-null    float64
 6   GC=F_Volume          756 non-null    float64
 7   GLD_Volume           754 non-null    float64
 8   SHY_Volume           754 non-null    float64
 9   TLT_Volume           754 non-null    float64
 10  ^VIX_Volatility      754 non-null    float64
 11  DX-Y.NYB_Volatility  756 non-null    float64
 12  TLT_Volatility       754 non-null    float64
 13  GC=F_Volatility      756 non-null    float64
 14  GLD_Volatility       754 non-null    floa

In [ ]:
# CSV 저장
# final_df.to_csv("../raw_data/external_environment_variable.csv")

### SPY_NAV(자산가치) 데이터 가져오기

In [ ]:
import pandas as pd

path = "../raw_data/navhist-us-en-spy.xlsx"

# 1. 엑셀 불러오기
df = pd.read_excel(path)

# 2. Date를 datetime으로 변환 (날짜가 아닌 건 NaT로 처리)
df["Date"] = pd.to_datetime(df["Date"], errors="coerce")

# 3. Date가 NaT인 행(문장 들어간 행 등) 제거
df = df.dropna(subset=["Date"])

# 4. 형식 'YYYY-MM-DD'로 바꾸기
df["Date"] = df["Date"].dt.strftime("%Y-%m-%d")

# 5. Date, NAV 두 컬럼만 남기기
nav_df = df[["Date", "NAV"]].copy()

# 6. NAV → SPY_NAV로 컬럼명 변경
nav_df = nav_df.rename(columns={"NAV": "SPY_NAV"})

print(nav_df.head())

         Date     SPY_NAV
0  2025-11-12  683.129392
1  2025-11-11  682.675112
2  2025-11-10  681.263847
3  2025-11-07  670.858117
4  2025-11-06  669.950151


In [ ]:
nav_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 5527 entries, 0 to 5526
Data columns (total 2 columns):
 #   Column   Non-Null Count  Dtype  
---  ------   --------------  -----  
 0   Date     5527 non-null   object 
 1   SPY_NAV  5527 non-null   float64
dtypes: float64(1), object(1)
memory usage: 129.5+ KB


In [ ]:
import pandas as pd

# 1. Date를 datetime으로 변환 (현재 'YYYY-MM-DD' 문자열 상태라고 가정)
nav_df["Date"] = pd.to_datetime(nav_df["Date"])

# 2. 날짜 범위 필터링 (2022-11-07 ~ 2025-11-07 포함)
start = pd.to_datetime("2022-11-07")
end   = pd.to_datetime("2025-11-07")

nav_df = nav_df[(nav_df["Date"] >= start) & (nav_df["Date"] <= end)]

# 3. Date 기준 오름차순 정렬
nav_df = nav_df.sort_values("Date")

# 4. 다시 'YYYY-MM-DD' 문자열로 쓰고 싶으면
nav_df["Date"] = nav_df["Date"].dt.strftime("%Y-%m-%d")

print(nav_df.head())


           Date     SPY_NAV
756  2022-11-07  379.817175
755  2022-11-08  381.939921
754  2022-11-09  374.104212
753  2022-11-10  394.806754
752  2022-11-11  398.441876


In [ ]:
nav_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 754 entries, 756 to 3
Data columns (total 2 columns):
 #   Column   Non-Null Count  Dtype  
---  ------   --------------  -----  
 0   Date     754 non-null    object 
 1   SPY_NAV  754 non-null    float64
dtypes: float64(1), object(1)
memory usage: 17.7+ KB


In [ ]:
# 5. CSV로 저장
nav_df.to_csv("../raw_data/spy_nav_only.csv", index=False, encoding="utf-8-sig")

### 기준금리 데이터 가져오기

In [ ]:
import pandas as pd

path = "../raw_data/기준금리.xlsx"

# 1. 엑셀 불러오기
df = pd.read_excel(path)

# 2. '날짜' 컬럼을 datetime으로 변환 후 'YYYY-MM-DD' 형식으로 변경
df["날짜"] = pd.to_datetime(df["날짜"], errors="coerce").dt.strftime("%Y-%m-%d")

# 3. '날짜' → 'Date' 로 컬럼 이름 변경
df = df.rename(columns={"날짜": "Date"})

# 4. Date, 금리 컬럼만 남기고 싶으면
rate_df = df[["Date", "금리"]].copy()

# 5. Date 기준 오름차순 정렬
rate_df = rate_df.sort_values("Date")

print(rate_df.head())

          Date    금리
29  2022-03-17  0.50
28  2022-05-05  1.00
27  2022-06-16  1.75
26  2022-07-28  2.50
25  2022-09-22  3.25


In [ ]:
rate_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 30 entries, 29 to 0
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   Date    30 non-null     object 
 1   금리      30 non-null     float64
dtypes: float64(1), object(1)
memory usage: 720.0+ bytes


In [ ]:
import pandas as pd

# 1. Date를 datetime으로 변환 (현재 'YYYY-MM-DD' 문자열 상태라고 가정)
rate_df["Date"] = pd.to_datetime(rate_df["Date"])

# 2. 날짜 범위 필터링 (2022-11-01 ~ 2025-11-07 포함)
start = pd.to_datetime("2022-11-01")
end   = pd.to_datetime("2025-11-07")

rate_df = rate_df[(rate_df["Date"] >= start) & (rate_df["Date"] <= end)]

# 3. Date 기준 오름차순 정렬
rate_df = rate_df.sort_values("Date")

# 4. 다시 'YYYY-MM-DD' 문자열로 쓰고 싶으면
rate_df["Date"] = rate_df["Date"].dt.strftime("%Y-%m-%d")

print(rate_df.head())

          Date    금리
24  2022-11-03  4.00
23  2022-12-15  4.50
22  2023-02-02  4.75
21  2023-03-23  5.00
20  2023-05-04  5.25


In [ ]:
print(rate_df)

          Date    금리
24  2022-11-03  4.00
23  2022-12-15  4.50
22  2023-02-02  4.75
21  2023-03-23  5.00
20  2023-05-04  5.25
19  2023-06-15  5.25
18  2023-07-27  5.50
17  2023-09-21  5.50
16  2023-11-02  5.50
15  2023-12-14  5.50
14  2024-02-01  5.50
13  2024-03-21  5.50
12  2024-05-02  5.50
11  2024-06-13  5.50
10  2024-08-01  5.50
9   2024-09-19  5.00
8   2024-11-08  4.75
7   2024-12-19  4.50
6   2025-01-30  4.50
5   2025-03-20  4.50
4   2025-05-08  4.50
3   2025-06-19  4.50
2   2025-07-31  4.50
1   2025-09-18  4.25
0   2025-10-30  4.00


In [ ]:
# 5. 필요하면 CSV로 저장
rate_df.to_csv("../raw_data/기준금리.csv", index=False, encoding="utf-8-sig")

### S&P 500 기업의 PER, PBR 데이터 불러오기

In [ ]:
import pandas as pd

path1 = "../raw_data/PBR.xlsx"

# 1. PBR 불러오기
df = pd.read_excel(path1)

# 2. Date를 datetime으로 변환 (날짜가 아닌 건 NaT로 처리)
df["Date"] = pd.to_datetime(df["Date"], errors="coerce")

# 3. Date가 NaT인 행(문장 들어간 행 등) 제거
df = df.dropna(subset=["Date"])

# 4. 형식 'YYYY-MM-DD'로 바꾸기
df["Date"] = df["Date"].dt.strftime("%Y-%m-%d")

# 5. Date, Value 두 컬럼만 남기기
PBR_df = df[["Date", "Value"]].copy()

print(PBR_df.head())

         Date   Value
0  2025-11-13     NaN
1  2025-09-30     NaN
2  2025-06-30    5.01
3  2025-03-31    4.69
4  2024-12-31    4.99


In [ ]:
PBR_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 105 entries, 0 to 104
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   Date    105 non-null    object
 1   Value   103 non-null    object
dtypes: object(2)
memory usage: 1.8+ KB


In [ ]:
import pandas as pd

# 1. Date를 datetime으로 변환 (현재 'YYYY-MM-DD' 문자열 상태라고 가정)
PBR_df["Date"] = pd.to_datetime(PBR_df["Date"])

# 2. 날짜 범위 필터링 (2022-09-01 ~ 2025-11-07 포함)
start = pd.to_datetime("2022-09-01")
end   = pd.to_datetime("2025-11-07")

PBR_df = PBR_df[(PBR_df["Date"] >= start) & (PBR_df["Date"] <= end)]

# 3. Date 기준 오름차순 정렬
PBR_df = PBR_df.sort_values("Date")

# 4. 다시 'YYYY-MM-DD' 문자열로 쓰고 싶으면
PBR_df["Date"] = PBR_df["Date"].dt.strftime("%Y-%m-%d")

print(PBR_df.head())

          Date   Value
13  2022-09-30    3.60
12  2022-12-31    3.75
11  2023-03-31    3.93
10  2023-06-30    4.19
9   2023-09-30    3.98


In [ ]:
PBR_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 13 entries, 13 to 1
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   Date    13 non-null     object
 1   Value   12 non-null     object
dtypes: object(2)
memory usage: 312.0+ bytes


In [ ]:
# 5. 필요하면 CSV로 저장
PBR_df.to_csv("../raw_data/SPY_PBR.csv", index=False, encoding="utf-8-sig")

In [ ]:
import pandas as pd

path2 = "../raw_data/PER.xlsx"

# PER 불러오기
df = pd.read_excel(path2)

# 2. Date를 datetime으로 변환 (날짜가 아닌 건 NaT로 처리)
df["Date"] = pd.to_datetime(df["Date"], errors="coerce")

# 3. Date가 NaT인 행(문장 들어간 행 등) 제거
df = df.dropna(subset=["Date"])

# 4. 형식 'YYYY-MM-DD'로 바꾸기
df["Date"] = df["Date"].dt.strftime("%Y-%m-%d")

# 5. Date, Value 두 컬럼만 남기기
PER_df = df[["Date", "Value"]].copy()

print(PER_df.head())

         Date Value
0  2025-11-13   NaN
1  2025-11-01   NaN
2  2025-10-01   NaN
3  2025-09-01   NaN
4  2025-08-01   NaN


In [ ]:
PER_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1860 entries, 0 to 1859
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   Date    1860 non-null   object
 1   Value   1854 non-null   object
dtypes: object(2)
memory usage: 29.2+ KB


In [ ]:
import pandas as pd

# 1. Date를 datetime으로 변환 (현재 'YYYY-MM-DD' 문자열 상태라고 가정)
PER_df["Date"] = pd.to_datetime(PER_df["Date"])

# 2. 날짜 범위 필터링 (2022-10-01 ~ 2025-11-07 포함)
start = pd.to_datetime("2022-10-01")
end   = pd.to_datetime("2025-11-07")

PER_df = PER_df[(PER_df["Date"] >= start) & (PER_df["Date"] <= end)]

# 3. Date 기준 오름차순 정렬
PER_df = PER_df.sort_values("Date")

# 4. 다시 'YYYY-MM-DD' 문자열로 쓰고 싶으면
PER_df["Date"] = PER_df["Date"].dt.strftime("%Y-%m-%d")

print(PER_df.head())

          Date    Value
38  2022-10-01    20.44
37  2022-11-01    22.07
36  2022-12-01    22.65
35  2023-01-01    22.82
34  2023-02-01    23.40


In [ ]:
PER_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 38 entries, 38 to 1
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   Date    38 non-null     object
 1   Value   33 non-null     object
dtypes: object(2)
memory usage: 912.0+ bytes


In [ ]:
# 5. 필요하면 CSV로 저장
PER_df.to_csv("../raw_data/SPY_PER.csv", index=False, encoding="utf-8-sig")

### 미국 GDP 데이터 불러오기

In [ ]:
# import requests
# import pandas as pd

# # 1. BEA API 기본 설정 -----------------------------
# API_KEY = "4B38B79F-E697-4FF8-B14D-1DCE719AA8AD"
# BASE_URL = "https://apps.bea.gov/api/data"

# # 2. 요청 파라미터 ---------------------------------
# params = {
#     "UserID": API_KEY,
#     "method": "GetData",
#     "DataSetName": "NIPA",
#     "TableName": "T10106",   # Table 1.1.6 (Real GDP, chained 2017 dollars)
#     "Frequency": "Q",
#     "Year": "ALL",
#     "ResultFormat": "JSON"
# }

# # 3. API 호출 --------------------------------------
# res = requests.get(BASE_URL, params=params)
# data = res.json()

# # 4. 정상 응답인지 체크 ------------------------------
# if "Error" in data["BEAAPI"]["Results"]:
#     raise ValueError(data["BEAAPI"]["Results"]["Error"])

# # 5. JSON → DataFrame 변환 -------------------------
# raw = data["BEAAPI"]["Results"]["Data"]
# df = pd.DataFrame(raw)

# # 6. 전체 GDP(LineNumber=1)만 추출 ------------------
# gdp = df[df["LineNumber"] == "1"].copy()

# # 7. GDP 문자열을 float으로 변환 ---------------------
# gdp["GDP_chained_2017_bil"] = (
#     gdp["DataValue"].str.replace(",", "", regex=False).astype(float)
# )

# # 8. 분기 문자열("2025Q2") → Date 변환 ---------------
# gdp["date"] = pd.PeriodIndex(gdp["TimePeriod"], freq="Q").to_timestamp(how="end")

# # 9. ─── 2022년 ~ 2025년 데이터만 필터링 ─────────────
# gdp = gdp[
#     (gdp["date"].dt.year >= 2022) &
#     (gdp["date"].dt.year <= 2025)
# ].copy()

# # 10. 최종 정리 -------------------------------------
# gdp = gdp[["date","GDP_chained_2017_bil"]]
# gdp = gdp.sort_values("date")

# print(gdp)

                             date  GDP_chained_2017_bil
300 2022-03-31 23:59:59.999999999            21932710.0
301 2022-06-30 23:59:59.999999999            21967045.0
302 2022-09-30 23:59:59.999999999            22125625.0
303 2022-12-31 23:59:59.999999999            22278345.0
304 2023-03-31 23:59:59.999999999            22439607.0
305 2023-06-30 23:59:59.999999999            22580499.0
306 2023-09-30 23:59:59.999999999            22840989.0
307 2023-12-31 23:59:59.999999999            23033780.0
308 2024-03-31 23:59:59.999999999            23082119.0
309 2024-06-30 23:59:59.999999999            23286508.0
310 2024-09-30 23:59:59.999999999            23478570.0
311 2024-12-31 23:59:59.999999999            23586542.0
312 2025-03-31 23:59:59.999999999            23548210.0
313 2025-06-30 23:59:59.999999999            23770976.0


In [ ]:
# # 1) date 컬럼명을 Date로 변경
# gdp = gdp.rename(columns={"date": "Date"})

# # 2) Date 컬럼을 연월일로만 변환 (YYYY-MM-DD)
# gdp["Date"] = pd.to_datetime(gdp["Date"]).dt.date

# # 3) 필요하면 다시 datetime 형식으로 (타임존 없이)
# gdp["Date"] = pd.to_datetime(gdp["Date"])

# print(gdp.head())

          Date  GDP_chained_2017_bil
300 2022-03-31            21932710.0
301 2022-06-30            21967045.0
302 2022-09-30            22125625.0
303 2022-12-31            22278345.0
304 2023-03-31            22439607.0


In [ ]:
# gdp.info()

<class 'pandas.core.frame.DataFrame'>
Index: 14 entries, 300 to 313
Data columns (total 2 columns):
 #   Column                Non-Null Count  Dtype         
---  ------                --------------  -----         
 0   Date                  14 non-null     datetime64[ns]
 1   GDP_chained_2017_bil  14 non-null     float64       
dtypes: datetime64[ns](1), float64(1)
memory usage: 336.0 bytes


In [ ]:
# # 5. 필요하면 CSV로 저장
# gdp.to_csv("/content/drive/MyDrive/예비프로젝트/버전2(SPY종가+환경변수)/raw_data/X변수_데이터/US_GDP.csv", index=False)